<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных-и-библиотек" data-toc-modified-id="Загрузка-данных-и-библиотек-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных и библиотек</a></span></li><li><span><a href="#Исследовательский-анализ-данных" data-toc-modified-id="Исследовательский-анализ-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследовательский анализ данных</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Анализ данных сервиса для чтения книг по подписке

**Описание проекта**

Компания приобрела крупный сервис для чтения книг по подписке. Необходимо проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Описание данных**

Таблица `books` содержит данные о книгах:

- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

Таблица `authors` содержит данные об авторах:

- author_id — идентификатор автора;
- author — имя автора.

Таблица `publishers` содержит данные об издательствах:

- publisher_id — идентификатор издательства;
- publisher — название издательства;

Таблица `ratings` содержит данные о пользовательских оценках книг:

- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

Таблица `reviews` содержит данные о пользовательских обзорах на книги:

- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя пользователя, написавшего обзор;
- text — текст обзора.

**План исследования**

1. Посчитать, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитать количество обзоров и среднюю оценку;
3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
4. Определить автора с самой высокой средней оценкой книг (только для книг с 50 и более оценками);
5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

## Загрузка данных и библиотек

Импортируем необходимые библиотеки.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

Установим соединение с базой данных.

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Выведем первые 5 строк всех таблиц на экран.

In [3]:
for table in ['books', 'authors', 'publishers', 'ratings', 'reviews']:
    query = '''SELECT * 
               FROM {} 
               LIMIT 5
            '''.format(table)
    print()
    print('Выведем первые 5 строк таблицы {} на экран'.format(table))
    con=engine.connect()
    display(pd.io.sql.read_sql(sql=text(query), con = con))


Выведем первые 5 строк таблицы books на экран


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268



Выведем первые 5 строк таблицы authors на экран


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd



Выведем первые 5 строк таблицы publishers на экран


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company



Выведем первые 5 строк таблицы ratings на экран


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2



Выведем первые 5 строк таблицы reviews на экран


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Исследовательский анализ данных

**1. Посчитаем, сколько книг вышло после 1 января 2000 года**

In [4]:
query = '''
           SELECT COUNT (book_id) AS books_count
           FROM books
           WHERE publication_date > '2000-01-01'
        '''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,books_count
0,819


После 1 января 2000 года количество выпущенных книг составило 819 шт.

**2. Для каждой книги посчитаем количество обзоров и среднюю оценку**

In [5]:
query = '''
           SELECT b.title, 
           COUNT(DISTINCT rv.review_id) AS reviews_count,
           ROUND(AVG(rt.rating), 2) AS avg_rating
           FROM books AS b
           LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
           LEFT JOIN reviews AS rv ON b.book_id = rv.book_id
           GROUP BY b.book_id
           ORDER BY b.title
        '''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,title,reviews_count,avg_rating
0,'Salem's Lot,2,3.67
1,1 000 Places to See Before You Die,1,2.50
2,13 Little Blue Envelopes (Little Blue Envelope...,3,4.67
3,1491: New Revelations of the Americas Before C...,2,4.50
4,1776,4,4.00
...,...,...,...
995,Wyrd Sisters (Discworld #6; Witches #2),3,3.67
996,Xenocide (Ender's Saga #3),3,3.40
997,Year of Wonders,4,3.20
998,You Suck (A Love Story #2),2,4.50


Количество обзоров на каждую из книг составляет от 0 до 7, а средняя оценка книг колеблется от 1.5 до 5.0.

**3. Определим издательство, которое выпустило наибольшее число книг толще 50 страниц (так мы исключим из анализа брошюры)**

In [6]:
query = '''
           SELECT p.publisher,
           COUNT (b.book_id) AS books_count
           FROM books AS b
           LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id
           WHERE num_pages > 50
           GROUP BY p.publisher
           ORDER BY COUNT(b.book_id) DESC
           LIMIT 1
        '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,books_count
0,Penguin Books,42


Издательством, которое выпустило наибольшее число книг толще 50 страниц является издательство Penguin Books.

**4. Определим автора с самой высокой средней оценкой книг (только для книг с 50 и более оценками)**

In [7]:
query = '''
           SELECT a.author, 
           ROUND(AVG(r.rating), 2) AS rating
           FROM books AS b
           LEFT JOIN ratings AS r ON b.book_id = r.book_id
           LEFT JOIN authors AS a ON b.author_id = a.author_id
           WHERE b.book_id IN (SELECT book_id
                               FROM ratings
                               GROUP BY book_id
                               HAVING COUNT (DISTINCT rating_id) >= 50)
           GROUP BY a.author_id
           ORDER BY(AVG(r.rating)) DESC
           LIMIT 1
        '''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,rating
0,J.K. Rowling/Mary GrandPré,4.29


У союза авторов J.K. Rowling/Mary GrandPré самые высокие средние оценки книг.

**5. Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок**

In [8]:
query = '''
           WITH tab_1 AS
           (SELECT rv.username,
           COUNT (DISTINCT rt.rating_id) AS rating_count,
           COUNT (DISTINCT rv.review_id) AS review_count
           FROM reviews AS rv
           LEFT JOIN ratings AS rt ON rv.username = rt.username
           GROUP BY rv.username
           HAVING COUNT (DISTINCT rt.rating_id) > 48)
           
           SELECT ROUND(SUM(review_count) / COUNT(*)) AS avg_reviews
           FROM tab_1
        '''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg_reviews
0,24.0


Пользователи, которые поставили больше 48 оценок, в среднем составляли 24 обзора на книги. 

## Вывод

Обзор базы данных сервиса для чтения книг показал, что в ней находится 1000 различных книг от 636 авторов и выпущенных 340 издательствами. 

По результатам исследования данных установлено:

- После 1 января 2000 года количество выпущенных книг составило 819 шт.
- Количество обзоров на каждую из книг составляет от 0 до 7, а средняя оценка книг колеблется от 1.5 до 5.0.
- Издательством, которое выпустило наибольшее число книг толще 50 страниц является издательство Penguin Books.
- У союза авторов J.K. Rowling/Mary GrandPré самые высокие средние оценки книг.
- Пользователи, которые поставили больше 48 оценок, в среднем составляли 24 обзора на книги.